In [ ]:
import pandas as pd
from sklearn import preprocessing

from gnnad.graphanomaly import GNNAD
from gnnad.plot import plot_test_anomalies, plot_predictions, plot_sensor_error_scores

def normalise(X, scaler_fn):
    scaler = scaler_fn.fit(X)
    return pd.DataFrame(scaler.transform(X), index=X.index, columns = X.columns)

In [ ]:
# read in training data
X_train = pd.read_csv('./examples/herbert_train.csv', index_col=0)
X_train.index = pd.to_datetime(X_train.index)

# read in test data
X_tmp = pd.read_csv('./examples/herbert_test.csv', index_col=0)
X_tmp.index = pd.to_datetime(X_tmp.index)
X_test = X_tmp.iloc[:,:8]
y_test = X_tmp.iloc[:,8:].apply(any, axis=1)

# normalise
X_test = normalise(X_test, preprocessing.StandardScaler())
X_train = normalise(X_train, preprocessing.StandardScaler())

# create ANOOMS dict for plotting
ANOMS = {'type1': {}}
X_test_anoms = X_tmp.iloc[:,8:]

for i in range(len(X_test_anoms.columns)):
    anom_col_name = X_test_anoms.columns[i]
    sensor_col_name = X_test.columns[i]
    anom_idxs = X_test_anoms[anom_col_name][X_test_anoms[anom_col_name]].index

    if len(anom_idxs) > 0:
        ANOMS['type1'][sensor_col_name] = anom_idxs

In [ ]:
# plot input data
plot_test_anomalies(X_test, ANOMS)

In [ ]:
# run model
model = GNNAD(threshold_type="max_validation", topk=6, slide_win=200)
fitted_model = model.fit(X_train, X_test, y_test)


In [ ]:
# model summary
fitted_model.summary()

In [ ]:
# GDN+, sensor thresholds
preds = fitted_model.sensor_threshold_preds(tau = 99)
fitted_model.print_eval_metrics(preds)


In [ ]:
# plot predictions
plot_predictions(fitted_model, X_test, ANOMS, preds = preds, figsize=(20, 20))


In [ ]:
plot_sensor_error_scores(fitted_model, X_test)